In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# looking at the top 100 disease-disease relations in a random set of 500 diseases
N_TOPRELATIONS = 100
N_DISEASE = 500

In [3]:
API='https://alpha.targetvalidation.org/api/1.1/'

def get_token(app_name='load-test',secret='1RT6L519zkcTH9i3F99OjeYn13k79Wep'):
    jwt = requests.get(API + 'public/auth/request_token',
                     params={'app_name':app_name,'secret':secret})
    return jwt.json()['token']

token = get_token()

In [4]:
# extract the score for one disease (eg.asthma)
def get_disease_relation(EFO):
    response = requests.get(API + 'private/relation/disease/' + EFO,
                        params={'size':N_TOPRELATIONS},
                        headers={'Auth-Token': token})
            
    return response.json()['data']

r = get_disease_relation('EFO_0000270')

In [22]:
def scoresfromjson(r): return pd.DataFrame([x['scores'] for x in r if x['object']['id'] != x['subject']['id']])

In [30]:
df = scoresfromjson(r)
df.hist()

In [38]:
# get more EFO codes with a hack
efo_codes = []
for t in ['ENSG00000141510','ENSG00000142168','ENSG00000073756','ENSG00000149311']:
    payload = {'target':t, 
                'facets':'false', 
                'size':500, 
                'direct':'true',
                'fields':'disease.id'}
    e = requests.get(API + '/public/association/filter', params= payload)
    efo_codes.extend([x['disease']['id'] for x in e.json()['data']])
    
print(len(efo_codes))
efo_codes_set = set(efo_codes)
print(len(efo_codes_set))

In [60]:
# select a random sample of EFO codes to check relationships for
import random

frames = []
for d in random.sample(efo_codes_set,N_DISEASE):
    try:
        frames.append(scoresfromjson(get_disease_relation(d)))
    except:
        continue

print(len(frames))
df = pd.concat(frames)
df.shape
# sample the distribution of the score

In [73]:
# Plot the distribution of all score
df.hist(color='k', alpha=0.5, bins=50,figsize=(14,8))